In [50]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import math
from tqdm.notebook import tqdm

In [51]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=130966#tab'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html, 'html.parser')

In [52]:
ifr_url = soup.find('iframe')['src']
url_front = url_base + ifr_url + '&page='
url_cnt = url_front+'1'
url_cnt

'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=130966&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'

In [53]:
html = urlopen(url_cnt)
soup = BeautifulSoup(html, 'html.parser')
count = soup.find('strong','total')
count = count.find('em').string     # get_text() 대신에 string
count = int(count.replace(',',''))
page_num = math.ceil(count/10)

In [54]:
star_score = []
reple = []
date = []
page_num = int(page_num/2)
for page in tqdm(range(1,page_num+1)):
    url = url_front+str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    uls = soup.find_all(class_='score_result')
    rest_list = uls[0].find_all('li')
    num = 0
    for rest in rest_list:
        date.append(rest.find(class_='score_reple').find('dt').find_all('em')[1].get_text())
        star_score.append(rest.find(class_='star_score').get_text().replace('\n',''))
        
        if rest.find(class_='_unfold_ment')!=None:
            reple.append(rest.find(class_='_unfold_ment').find('a')['data-src'])
        else:
            reple.append(rest.find(id='_filtered_ment_'+str(num)).get_text().strip())
        
        num += 1        

In [55]:
df = pd.DataFrame({'star_score':star_score,'reple':reple,'date':date})
df['date'] = pd.to_datetime(df['date'])
# df.sort_values(by='date', ascending = True, inplace=True)
df

,star_score,reple,date
0,10,좀비가 무서운게아니라 사람이무섭다는걸 알려주는영화,2016-07-20 05:57:00
1,10,그냥 한국형 좀비물이 어색하지 않았다는 거에 큰 점수를 주며 앞으로의 한국영화를 기...,2016-07-20 11:56:00
2,10,버스회사 상무라는 사람이 진심 역대급 민폐캐릭터인듯개짜증나고 생존력도 좋음,2016-07-20 00:00:00
3,10,생각보다 너무 재밌었다,2016-07-20 01:11:00
4,6,영화가 너무 비현실적이네요. 예를들어 공유 얼굴이라던가 공유 얼굴 같은 부분이...,2016-07-20 07:30:00
...,...,...,...
28995,9,스릴도 있고 여름휴가동안 더위를 잊게 해주는 보기좋은 영화중에 하나라 생각듭니다.재...,2016-07-31 21:38:00
28996,10,영화를 너무 잘만들다 보니 보는내내 짜증이 너무 났습니다. 실제로 저런일이 일어...,2016-07-31 21:38:00
28997,8,재밌게 봤어요 스토리가 예상이 되었지만 무섭고 슬펐어요,2016-07-31 21:37:00
28998,6,무섭기만 했습니다 해피엔딩도 아니고ㅠ,2016-07-31 21:36:00


In [56]:
df.to_csv('movie_grade_trainToBusan.csv',sep=',',encoding="utf-8")

In [61]:
from wordcloud import WordCloud

In [65]:
wordcloud = WordCloud.generate_from_dataframe(df)

AttributeError: type object 'WordCloud' has no attribute 'generate_from_dataframe'